# create_detect_evaluate.ipynb

In [2]:
import fiftyone as fo
from ultralytics import YOLO

I installed mongodb using:

https://www.cherryservers.com/blog/install-mongodb-ubuntu-2404

Could not connect fiftyone to mongodb because of authentication issues.
So I disabled security:autorization in /etc/mongod.conf

In [ ]:
# Delete an existing FiftyOne dataset to force rebuilding. Normally set to False. 
# DELETE_DATASET = True

# FiftyOne dataset name
DATASET_NAME = 'CRB002s'

# YOLO dataset with train,val splits
DATASET_PATH = '/home/aubrey/Desktop/crb-damage-detection-models/datasets/CRB002s'

# Path to YOLOv11 model
LATEST_MODEL = '/home/aubrey/Desktop/crb-damage-detection-models/models/CRB002/weights/CRB002.pt'

# Model name
MODEL_NAME = 'CRBOO2'

In [4]:
# List all local datasets by name
# Reports error 26 on first try, indicating a problem opening the Mongo database.
# The following hack runs the code a second time. Seems to work well.
# Note: A dataset can be deleted using fo.delete_dataset("dataset name")
try:
    dataset_list = fo.list_datasets()
except:
    dataset_list = fo.list_datasets()
dataset_list

['CRB002s', 'quickstart']

In [15]:
dataset_names = fo.list_datasets()
for name in dataset_names:
    fo.delete_dataset(name)
fo.list_datasets()

[]

In [16]:
# Create the dataset if it does not already exist
name = DATASET_NAME
# if dataset exists, load it
if name in fo.list_datasets():   # load existing dataset
    print(f'Loading existing FiftyOne dataset: {name}')
    dataset = fo.load_dataset(name) 
else:   
    
    # create a new FiftyOne dataset
    print(f'---- Creating new FifyOne datase: {name}')
    dataset_dir = DATASET_PATH
    splits = ["train", "val"]   
    dataset = fo.Dataset(name, persistent=True)   # create persistant dataset
    for split in splits:
        dataset.add_dir(
            dataset_dir=dataset_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            split=split,
            tags=split,
        )
    print(dataset)
    print(f'\ntags: {dataset.distinct("tags")}\n\n')
    
    # Add detections  
    print(f'--- Adding detections using YOLOv11 model:\n{LATEST_MODEL}')  
    model = YOLO(LATEST_MODEL)
    dataset.apply_model(model, label_field=MODEL_NAME, progress=True)
    # print summary info about the dataset
    print(dataset)
    print(f'\ntags: {dataset.distinct("tags")}\n\n')
    
    # Evaluate detections
    print('--- Running dataset.evaluate_detections')
    dataset.evaluate_detections(MODEL_NAME, gt_field='ground_truth', eval_key='eval')
    
    # print summary info about the dataset
    print(dataset)
    print(f'\ntags: {dataset.distinct("tags")}')

    # dataset.evaluate_detections(
    #     pred_field='instances', 
    #     gt_field='ground_truth', 
    #     # eval_key=None, 
    #     # classes=None, 
    #     # missing=None, 
    #     # method=None, 
    #     # iou=0.5, 
    #     use_masks=True, 
    #     use_boxes=False, 
    #     classwise=True, 
    #     dynamic=True, 
    #     progress=True,
    # )    
# print summary info about the dataset

---- Creating new FifyOne datase: CRB002s
 100% |███████████████████| 60/60 [110.9ms elapsed, 0s remaining, 540.8 samples/s]    
 100% |███████████████████| 23/23 [36.4ms elapsed, 0s remaining, 631.7 samples/s]    
Name:        CRB002s
Media type:  image
Num samples: 83
Persistent:  True
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

tags: ['train', 'val']


--- Adding detections using YOLOv11 model:
/home/aubrey/Desktop/crb-damage-detection-models/models/CRB002/weights/CRB002.pt
 100% |█████████

In [17]:
# session.show()
session = fo.launch_app(dataset, auto=False)
print('At this point, the dataset should be available for exploration at http://localhost:5151')

Session launched. Run `session.show()` to open the App in a cell output.
At this point, the dataset should be available for exploration at http://localhost:5151



Could not connect session, trying again in 10 seconds

